# Ejercicio 3

3. Genera datos aleatorios, al menos $m = 1000$ observaciones, de un modelo de regresión lineal 
$y=\beta_{0}+\beta_{1}x_1+\dots+\beta_{n}x_n+e$ predefinido con al menos $n = 100$ variables. Considera que los coeficientes de la regresión $\beta_{0}, ...\beta_{i}, ..., \beta_{n}$ son enteros donde:
$-5 <= \beta_{i} <= -5$ para todo $i$.

Considera también que los errores (residuos) son normales $e\sim N(0,\sigma^{2})$ e independientes.



In [ ]:
%matplotlib notebook
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from numpy.linalg import inv
import time
%matplotlib inline

In [ ]:
m = 1000 ## m obeservaciones
n = 100

np.random.seed(42)

x_0, x_1 = np.ones([m, 1]), np.random.uniform(0, 10, ([m, n])) #beta0 = 1
X = np.concatenate([x_0, x_1],axis=1)

beta = np.random.randint(-5, 5, size=([n + 1, 1]))

error = np.random.normal(0, 1, (m, 1))#error normal aleatorio

Y = np.dot(X,beta) + error

In [ ]:
plt.scatter(X[:,1], Y),plt.xlabel('X'),plt.ylabel('Y')

a) Estimar el valor de los parámetros de la regresión aplicando la solución analítica del problema de mínimos cuadrados.

  \begin{align*}
\text{minimize}\quad & ||y-X\beta||_2^2
\end{align*}

La solución explícita es: $\beta_{ls}=(X^T X)^{-1}X^T y$

In [ ]:
def explicit_solution(X, Y):    
    return np.dot(np.dot(inv(np.dot((X.T),X)),X.T),Y)

explicit_beta = explicit_solution(X, Y)
explicit_beta.T

b) Estimar el valor de los coeficientes de la regresión por mínimos cuadrados usando la herramienta minimize del paquete de Python **Scipy.optimize**. Prueba al menos cuatro solvers diferentes y compara su eficiencia en términos de: número de iteraciones totales, número de evaluaciones de la función objetivo, gradiente y hesiano, así como el tiempo de
cómputo total.

In [ ]:
def least_squared(beta, X, Y):
    beta = np.matrix(beta)
    z = Y - X * (beta.T)
    return np.dot(z.T,z)

#gradient
def derivative(beta, X, Y):
    beta = np.matrix(beta)    
    pp = -2 * np.dot((Y - np.dot(X,beta.T)).T,X)
    aa = np.squeeze(np.asarray(pp))
    return aa

def hess(beta, X, Y):
    return 2 * np.dot(np.transpose(X),X)

def linear_regression(beta, X):
    print('beta', beta)
    beta = np.matrix(beta)
    return np.dot(X,beta.T)

In [ ]:
# - 'Nelder-Mead' :ref:`(see here) <optimize.minimize-neldermead>`
# - 'Powell'      :ref:`(see here) <optimize.minimize-powell>`
# - 'CG'          :ref:`(see here) <optimize.minimize-cg>`
# - 'BFGS'        :ref:`(see here) <optimize.minimize-bfgs>`
# - 'Newton-CG'   :ref:`(see here) <optimize.minimize-newtoncg>`
# - 'L-BFGS-B'    :ref:`(see here) <optimize.minimize-lbfgsb>`
# - 'TNC'         :ref:`(see here) <optimize.minimize-tnc>`
# - 'COBYLA'      :ref:`(see here) <optimize.minimize-cobyla>`
# - 'SLSQP'       :ref:`(see here) <optimize.minimize-slsqp>`
# - 'trust-constr':ref:`(see here) <optimize.minimize-trustconstr>`
# - 'dogleg'      :ref:`(see here) <optimize.minimize-dogleg>`
# - 'trust-ncg'   :ref:`(see here) <optimize.minimize-trustncg>`
# - 'trust-exact' :ref:`(see here) <optimize.minimize-trustexact>`
# - 'trust-krylov' :ref:`(see here) <optimize.minimize-trustkrylov>`

In [ ]:
solvers = ['BFGS', 'Newton-CG', 'L-BFGS-B', 'SLSQP']
results = {key:{} for key in solvers}

beta_ls0 = np.zeros(n+1)
for solver in solvers:
    output = minimize(fun=least_squared, x0=beta_ls0, args=(X,Y), method=solver, hess=hess, jac=derivative, options={'disp': True})
    print(output)
    print('---------------------')
    results[solver]['success'] = output.success
    results[solver]['status'] = output.status
    results[solver]['message'] = output.message
    results[solver]['nit'] = output.nit    
    results[solver]['fun'] = output.fun
    results[solver]['x'] = output.x

In [ ]:
import pandas as pd
df = pd.DataFrame(data=results)
df = df.transpose()
df

* Solver: BFGS

In [ ]:
minimize(fun=least_squared, x0=beta_ls0, args=(X,Y), method='BFGS', jac=derivative, options={'disp': True})

* Newton-CG

In [ ]:
minimize(fun=least_squared, x0=beta_ls0, args=(X,Y), method='Newton-CG', jac=derivative, hess=hess, options={'disp': True})

* L-BFGS-B

In [ ]:
minimize(fun=least_squared, x0=beta_ls0, args=(X,Y), method='L-BFGS-B', jac=derivative, options={'disp': True})

* SLSQP

In [ ]:
minimize(fun=least_squared, x0=beta_ls0, args=(X,Y), method='SLSQP', jac=derivative, options={'disp': True})

c) Estimar el valor de los coeficientes de la regresión por mínimos cuadrados implementando manualmente:

#### Método del Gradiente

 $\rightarrow$Partimos de un punto inicial $x_0$

$\rightarrow$ Calculamos la dirección de descenso $p_k$

$\rightarrow$ Mientras que estemos lejos de la solución, calculamos la longitud de paso $\alpha_k>0$

$\rightarrow$ Movemos el punto:
$$x_{k+1} = x_k + \alpha_k\ p_k$$
Hasta la convergencia a una solución local.

In [ ]:
# (a,b)=X.shape
# beta0 = np.zeros(b) #punto inicial
# alpha=0.00005
# n_iter=10000 #máximo número de iteraciones
# OF_iter=np.zeros(n_iter)
# i=0;
# tol=1000;
# epsilon=1e-3;

# time_start = time.time()
# while (i <= n_iter - 2) and (tol > epsilon):
#    ...

time_start = time.time()

def descent(X, alpha=0.00005, epsilon=1e-3, n_iter=100, function_pk=derivative):    
    beta_k = np.matrix(np.zeros(X.shape[1]))
    k, norm = 0, 1
    sigma, beta, s = 0.0005, 0.5, 1
    alpha = s
    while(norm > epsilon and k < n_iter):
        k += 1
        grad = function_pk(beta_k, X, Y)        
        p_k = -1 * grad
        exp = 0
        while(linear_regression(beta_k + alpha * p_k, X) > 
              linear_regression(beta_k, X) + sigma*alpha*np.dot(p_k.T,grad)).all():            
            alpha = beta**exp * s
            exp += 1
            print(exp)
            print(alpha)
        beta_k = beta_k + alpha * p_k
        norm = np.linalg.norm(p_k, ord = 2)        
    return beta_k

def gradient_descent(X, alpha=0.00005,epsilon=1e-3, n_iter=100):
    return descent(X, alpha, epsilon, n_iter, function_pk=derivative)

In [ ]:
betas = gradient_descent(X)
betas

#### Método de Newton

In [ ]:
def newton(X, alpha=0.00005,epsilon=1e-3, n_iter=1000):
    return descent(X, alpha, epsilon, n_iter, function_pk=hess)

In [ ]:
b = newton(X)
b

#### Método de Quasi-Newton

In [ ]:
def quasi_newton(X, alpha=0.00005,epsilon=1e-3, n_iter=1000):
    beta_ls0 = np.zeros(n+1)
    beta_k = hess(beta_ls0, X, Y)
    k, norm = 0, 1   
    while(norm > epsilon and k < n_iter):
        p_k = descent_quasi_newton(beta_k, X, Y)
        beta_k = beta_k - alpha * p_k
        norm = np.linalg.norm(p_k, ord = 2)
        k += 1
    return beta_k   
    

In [ ]:
quasi_newton(X)

In [ ]:
def descent_quasi_newton(beta_k, X, Y):
    return -1 * np.dot(inv(beta_k), derivative(beta_k, X, Y))    